#  Tensorflow Timeline Analysis on Model Zoo Benchmark between Intel optimized and stock Tensorflow

This jupyter notebook will help you evaluate performance benefits from Intel-optimized Tensorflow on the level of Tensorflow operations via several pre-trained models from Intel Model Zoo. The notebook will show users a bar chart like the picture below for the Tensorflow operation level performance comparison. The red horizontal line represents the performance of Tensorflow operations from Stock Tensorflow, and the blue bars represent the speedup of Intel Tensorflow operations. The operations marked as "mkl-True" are accelerated by oneDNN a.k.a MKL-DNN, and users should be able to see a good speedup for those operations accelerated by oneDNN. 
> NOTE : Users need to get Tensorflow timeline json files from other Jupyter notebooks like benchmark_perf_comparison
  first to proceed this Jupyter notebook.

> NOTE: Users could also compare elapsed time of TF ops among any two different TF timeline files.


<img src="images\compared_tf_op_duration_ratio_bar.png" width="700">

The notebook will also show users two pie charts like the picture below for elapsed time percentage among different Tensorflow operations.   
Users can easily find the Tensorflow operation hotspots in these pie charts among Stock and Intel Tensorflow.

<img src="images\compared_tf_op_duration_pie.png" width="700">

# Get Platform Information 

In [ ]:
# ignore all warning messages
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from profiling.profile_utils import PlatformUtils
plat_utils = PlatformUtils()
plat_utils.dump_platform_info()

#  Section 1: TensorFlow Timeline Analysis
## Prerequisites

In [ ]:
!pip install cxxfilt

%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)

## List out the Timeline folders

First, list out all Timeline folders from previous runs.

In [ ]:
import os
filenames= os.listdir (".") 
result = []
keyword = "Timeline"
for filename in filenames: 
    if os.path.isdir(os.path.join(os.path.abspath("."), filename)): 
        if filename.find(keyword) != -1:
                result.append(filename)
result.sort()

index =0 
for folder in result:
    print(" %d : %s " %(index, folder))
    index+=1

## Select a Timeline folder from previous runs
#### ACTION: Please select one Timeline folder and change FdIndex accordingly

In [ ]:
# use the "FD_INDEX" environment variable value if it exists.
import os
env_fd_index=os.environ.get('FD_INDEX', '')
if env_fd_index != '':
    FdIndex= int(env_fd_index)
else:
    ## USER INPUT
    FdIndex= int(input('Input a index number of a folder: '))

List out all Timeline json files inside Timeline folder.

In [ ]:
import os
TimelineFd = result[FdIndex]
print(TimelineFd)
datafiles = [TimelineFd +os.sep+ x for x in sorted(os.listdir(TimelineFd), reverse=True) if '.json' == x[-5:]]
print(datafiles)
if len(datafiles) is 0:
    print("ERROR! No json file in the selected folder. Please select other folder.")
elif len(datafiles) is 1:
    print("WARNING! There is only 1 json file in the selected folder. Please select other folder to proceed Section 1.2.")

> **Users can bypass below Section 1.1 and analyze performance among Stock and Intel TF by clicking the link : [Section 1_2](#section_1_2).**

<a id='section_1_1'></a>
## Section 1.1: Performance Analysis for one TF Timeline result
### Step 1: Pick one of the Timeline files
#### List out all the Timeline files first


In [ ]:
index = 0
for file in datafiles:
    print(" %d : %s " %(index, file))
    index+=1

#### ACTION: Please select one timeline json file and change file_index accordingly

In [ ]:
## USER INPUT
# use the "FILE_INDEX" environment variable value if it exists.
import os
env_file_index=os.environ.get('FILE_INDEX', '')
if env_file_index != '':
    file_index= int(env_file_index)
else:
    ## USER INPUT
    file_index= int(input('Input a index number of a file: '))

fn = datafiles[file_index]
tfile_prefix = fn.split('_')[0]
tfile_postfix = fn.strip(tfile_prefix)[1:]
fn

### Step 2: Parse timeline into pandas format

In [ ]:
from profiling.profile_utils import TFTimelinePresenter
tfp = TFTimelinePresenter(True)
timeline_pd = tfp.postprocess_timeline(tfp.read_timeline(fn))
timeline_pd = timeline_pd[timeline_pd['ph'] == 'X']

### Step 3: Sum up the elapsed time of each TF operation

In [ ]:
tfp.get_tf_ops_time(timeline_pd,fn,tfile_prefix)

### Step 4: Draw a bar chart for elapsed time of TF ops 

In [ ]:
filename= tfile_prefix +'_tf_op_duration_bar.png'
title_=tfile_prefix +'TF : op duration bar chart'
ax=tfp.summarize_barh(timeline_pd, 'arg_op', title=title_, topk=50, logx=True, figsize=(10,10))
tfp.show(ax,'bar')

### Step 5: Draw a pie chart for total time percentage of TF ops 

In [ ]:
filename= tfile_prefix +'_tf_op_duration_pie.png'
title_=tfile_prefix +'TF : op duration pie chart'
timeline_pd_known = timeline_pd[ ~timeline_pd['arg_op'].str.contains('unknown') ]
ax=tfp.summarize_pie(timeline_pd_known, 'arg_op', title=title_, topk=50, logx=True, figsize=(10,10))
tfp.show(ax,'pie')
ax.figure.savefig(filename,bbox_inches='tight')

<a id='section_1_2'></a>
## Section 1.2: Analyze TF Timeline results between Stock and Intel Tensorflow
> NOTE : Users could also compare elapsed time of TF ops among any two different TF timeline files.

### Speedup from oneDNN among different TF operations

### Step 1: Select  one Intel and one Stock TF timeline files for analysis
> NOTE: Users could also pick any two different TF timeline files.

#### List out all timeline files in the selected folder

In [ ]:
if len(datafiles) is 1:
    print("ERROR! There is only 1 json file in the selected folder.")
    print("Please select other Timeline folder from beginnning to proceed Section 1.2.")

for i in range(len(datafiles)):
    print(" %d : %s " %(i, datafiles[i]))

#### ACTION: Please select one timeline file as a perfomance baseline and the other as a comparison target
put the related index for your selected timeline file.
In general, please put stock_timeline_xxxxx as the baseline.

In [ ]:
# perfomance baseline 
# use the "BASELINE_INDEX" environment variable value if it exists.
import os
env_baseline_index=os.environ.get('BASELINE_INDEX', '')
if env_baseline_index != '':
    Baseline_Index= int(env_baseline_index)
else:
    ## USER INPUT
    Baseline_Index= int(input('Input a index number of a Performance Baseline: '))
# comparison target
Comparison_Index = 0 if Baseline_Index else 1

#### List out two selected timeline files

In [ ]:
selected_datafiles = []
selected_datafiles.append(datafiles[Baseline_Index])
selected_datafiles.append(datafiles[Comparison_Index])
print(selected_datafiles)

### Step 2: Parsing timeline results into CSV files

In [ ]:
%matplotlib agg
from profiling.profile_utils import TFTimelinePresenter
csvfiles=[]
percentage_filename = ''

tfp = TFTimelinePresenter(True)
for fn in selected_datafiles:
    if fn.find('/'):
        fn_nofd=fn.split('/')[1]
    else:
        fn_nofd=fn
    tfile_name= fn_nofd.split('.')[0]
    tfile_prefix = fn_nofd.split('_')[0]
    tfile_postfix = fn_nofd.strip(tfile_prefix)[1:]
    csvpath = TimelineFd +os.sep+tfile_name+'.csv'
    print(csvpath)
    csvfiles.append(csvpath)
    timeline_pd = tfp.postprocess_timeline(tfp.read_timeline(fn))
    timeline_pd = timeline_pd[timeline_pd['ph'] == 'X']
    sitems, percentage_filename = tfp.get_tf_ops_time(timeline_pd,fn,tfile_prefix)

#### The pie chart for elapsed time of  oneDNN operations from Intel TF 

In [ ]:
%matplotlib inline
if percentage_filename != '':
    print(percentage_filename)
    tfp.plot_pie_chart(percentage_filename, 'mkl_percentage')

### Step 3: Pre-processing for the two CSV files

In [ ]:
import os
import pandas as pd

csvarray = []
csvfilenames= []
for csvf in csvfiles:
    print("read into pandas :",csvf)
    a = pd.read_csv(csvf)
    csvarray.append(a)
    if csvf.find(os.sep) > 0:
        csvfilenames.append(csvf.split(os.sep)[-1])
    else:
        csvfilenames.append(csvf)

a = csvarray[0]
b = csvarray[1]

In [ ]:
# Find tags among CSV files
tags=[]
from profiling.profile_utils import PerfPresenter
perfp=PerfPresenter()
tag0, tag1 = perfp.get_diff_from_csv_filenames(csvfilenames[0][:-4],csvfilenames[1][:-4])
tags = [tag0, tag1]
print('tags : ',tags)

### Step 4: Merge two CSV files and caculate the speedup accordingly

#### Merge two csv files

In [ ]:
import os
import pandas as pd
fdir='merged'
if not os.path.exists(fdir):
    os.mkdir(fdir)

fpaths=[]
fpaths.append(fdir+os.sep+'merged.csv')
fpaths.append(fdir+os.sep+'diff_'+tags[0]+'.csv')
fpaths.append(fdir+os.sep+'diff_'+tags[1]+'.csv')
#merged=tfp.merge_two_csv_files(fpath,a,b)
merged=tfp.merge_two_csv_files_v2(fpaths, a, b, tags)


### Compare common operations among those two csv files

In [ ]:
print("Compare common operations between ", tags)
merged_df = pd.read_csv(fpaths[0])
merged_df

#### The unique Tensorflow operations from the first csv/Timline file

In [ ]:
%matplotlib inline
print("Operations are only in", tags[0], " run")
extra1 = pd.read_csv(fpaths[1])
extra1

#### The unique Tensorflow operations from the second csv/Timline file

In [ ]:
print("Operations are only in", tags[1], " run")
extra2 = pd.read_csv(fpaths[2])
extra2

### Step 5: Draw a bar chart for elapsed time of common TF ops among stock TF and Intel TF

> NOTE: Users could also compare elapsed time of TF ops among any two different TF timeline files.


In [ ]:
%matplotlib inline
print(fpaths[0])
tfp.plot_compare_bar_charts(fpaths[0], tags=tags)
tfp.plot_compare_ratio_bar_charts(fpaths[0], tags=['','oneDNN ops'], max_speedup=20)

### Step 6: Draw pie charts for elapsed time of TF ops among stock TF and Intel TF

> NOTE: Users could also compare elapsed time of TF ops among any two different TF timeline files.

We will have following pie charts in sequence:
1. the pie chart for elpased time of TF ops from stock TF or the first csv/Timeline file
2. the pie chart for elpased time of unique TF ops from stock TF or the first csv/Timeline file
3. the pie chart for elpased time of TF ops from Intel TF or the second csv/Timeline file
4. the pie chart for elpased time of unique TF ops from Intel TF or the second csv/Timeline file
5. the pie chart for elpased time of common TF ops among stock & Intel TF or two csv/Timeline files


#### The pie chart for elapsed time of TF ops from Stock TF or the first csv/Timline file
understand which TF operations spend most of time.

In [ ]:
tfp.plot_pie_chart(csvfiles[0], tags[0])

#### The pie chart for elapsed time of  unique TF operations from Stock TF or the first csv/Timline file
understand if there is any unique TF operation.

In [ ]:
tfp.plot_pie_chart(fpaths[1], tags[0])

#### The pie chart for elapsed time of TF ops from Intel TF or the second csv/Timline file
understand which TF operations spend most of time.

In [ ]:
tfp.plot_pie_chart(csvfiles[1], tags[1])

#### The pie chart for elapsed time of  unique TF operations from Intel TF or the seond csv/Timline file
understand if there is any unique TF operation.

In [ ]:
tfp.plot_pie_chart(fpaths[2], tags[1])

#### The pie chart for elapsed time of common TF ops among Stock & Intel TF or  two csv/Timline files
understand top hotspots differences among Stock & Intel TF or two csv/Timeline files.

In [ ]:
tfp.plot_compare_pie_charts(fpaths[0], tags=tags)